In [126]:
%matplotlib widget

In [127]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [128]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [185]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e) 

Physical devices cannot be modified after being initialized


In [129]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

# input json name

In [651]:
file_parameters['watershed'] = 'geum1'

In [652]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [653]:
pd.set_option('display.max_columns', 1000)

In [654]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
model_input_path = file_parameters['watershed']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [655]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [656]:
gain_epochs , rnn_epochs

(2000, 15)

# run file

In [659]:
run_num = range(len(folder))


run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


# Gain Training Flag

In [660]:
__GAIN_TRAINING__ = False

In [661]:
month=12
day=31
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times , month, day= make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], 
                               interpolation=interpolation_option[idx],first_file_no=i, month=month, day=day)

    df_all, train_mean, train_std, df = normalize(df)


    

    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx']]
data/geum/자동/현도_2013.xlsx
data/geum/자동/현도_2014.xlsx
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-9-19 23:00
data/geum/자동/대청호_2013.xlsx
data/geum/자동/대청호_2014.xlsx
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-9-19 23:00
MissData :  save/  miss :  (6828, 12)
1/1 [============================

In [662]:
print(real_df_all.shape)

(67656, 24)


In [663]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

In [664]:
df[0]

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값,Day sin,Day cos,Year sin,Year cos
0,-0.999372,-0.442086,0.120697,-2.245811,-0.282605,0.124212,-0.331774,NaN,-1.136911e-11,1.414208,-0.020807,1.441032
1,-1.003368,-0.449395,0.113439,-2.245811,-0.277818,0.002180,-0.364482,NaN,3.660241e-01,1.366020,-0.019791,1.441022
2,-1.010983,-0.454137,0.108640,-2.245811,-0.284093,0.053973,-0.339703,NaN,7.071042e-01,1.224740,-0.018775,1.441011
3,-1.017229,-0.454887,0.102267,-2.245811,-0.293047,0.126508,-0.339703,NaN,9.999963e-01,0.999996,-0.017759,1.440999
4,-1.022970,-0.454861,0.102028,-2.245811,-0.284465,0.199343,-0.389262,NaN,1.224740e+00,0.707104,-0.016743,1.440986
...,...,...,...,...,...,...,...,...,...,...,...,...
67651,0.945553,-1.414106,-1.420009,-3.910879,1.294976,NaN,3.270158,-0.530224,-1.366020e+00,0.366024,-1.425877,-0.243100
67652,0.926016,-1.468988,-1.522608,-3.967409,1.253629,NaN,3.178970,-0.554405,-1.224740e+00,0.707104,-1.426072,-0.242106
67653,0.929272,-1.468988,-1.535908,-3.972548,1.286691,NaN,3.203749,-0.562147,-9.999963e-01,0.999996,-1.426267,-0.241113
67654,0.922760,-1.468988,-1.513739,-3.972548,1.232969,NaN,3.476323,-0.563633,-7.071042e-01,1.224740,-1.426461,-0.240119


# slice DF

In [665]:
# fake_df=real_df_all.iloc[:-2000, :]

In [684]:
train_df, val_df,test_df,test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)

In [685]:
val_df.shape, test_df.shape, train_df.shape

((6765, 24), (13532, 24), (47359, 24))

In [686]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (47359, 24) val_df.shape :  (6765, 24) test_df.shape: (13532, 24)


# target column

In [687]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "tn"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  tn
target_col_idx :  5
out_num_features :  1


In [688]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

# if __RNN_TRAINING__:
#     if not os.path.exists('save/' + model_input_path):
#         os.makedirs('save/' + model_input_path)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + model_input_path+ '/' + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/geum/models/geum1/nitrogen/


In [689]:
plt.figure()
plt.plot(real_df_all.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [690]:
plt.figure()
plt.plot(val_df.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 모델 학습

In [691]:
__RNN_TRAINING__ = False
continue_train = False

In [692]:
from tensorflow.keras.callbacks import Callback

In [693]:
class checkpoint_save(Callback):
    def __init__(self, model=None, save_path=None, val_nse=None):
        super(Callback, self).__init__()        
#         print('val_nse in checkpoint = ' , val_nse)
        self.best_score = val_nse
        self.model = model
        self.save_path=save_path

    def on_epoch_end(self, epoch, logs={}):
        #current_score = logs.get('val_accuracy')
#         current_score = logs.get('accuracy')
        current_score = logs.get('val_nse')
#         print('current_score = ',current_score)
#         print('self.best_score = ', float(val_nse[2]))
#         print(type(float(val_nse[2])))
#         print('current_score = ', current_score)
        
#         print(type(current_score))
        if current_score > self.best_score:
            print('*****save model: nse from %.3f to %.3f*****' %(self.best_score, current_score) )
            self.best_score = current_score            
            self.model.save_weights(self.save_path)
            

In [694]:
def compile_and_fit(model, window, patience=1000, epochs=400, save_path=None, val_nse=-100):
    
    print('val_nse= ', val_nse)

    checkpoint = checkpoint_save(model=model, save_path=save_path, val_nse = val_nse)
    
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         save_path, monitor='val_nse', verbose=1,
#         save_best_only=True, 
#         save_weights_only= True,
#         mode='max', period=1)

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min')

    model.compile(
        loss=tf.losses.MeanSquaredError(),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.MeanAbsoluteError(), nse])

    history = model.fit(
        #window.train, epochs=epochs,
        window.train, epochs=epochs, steps_per_epoch=10,
        validation_data=window.val,
        callbacks=[early_stopping, checkpoint])
    return history

In [695]:
def model_gru(OUT_STEPS=24*5, out_num_features=1, window=None, epochs=100, training_flag=False, 
              checkpoint_path="save/", continue_train = False):
    model = GRUModel(OUT_STEPS, out_num_features)
    #checkpoint_path = "save/gru_model.ckpt"
#   model.load_weights(checkpoint_path)
    if training_flag == True:
        model_evaluate_value = 0
        if continue_train :
            print("continue")
            model.load_weights(checkpoint_path)
            compile(model)
            model_evaluate_value = model.evaluate(window.val)
            print('model_evaluate_value = ', model_evaluate_value)
            history = compile_and_fit(model,window, epochs=epochs, save_path = checkpoint_path,
                                      val_nse=model_evaluate_value[2])
            
        else:
            history = compile_and_fit(model,window, epochs=epochs, save_path = checkpoint_path)                
        model.load_weights(checkpoint_path)
    else:
        model.load_weights(checkpoint_path)
        compile(model)
    return model

In [696]:
rnn_epochs = 15

# print('Linear Running....')
# multi_linear_model = model_multi_linear(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt", continue_train=continue_train)

# print('ELMAN Running....')
# elman_model = model_elman(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt", continue_train=continue_train)

print('GRU Running....')
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=continue_train)

# print('LSTM Running....')
# multi_lstm_model = model_multi_lstm(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt", continue_train=continue_train)

# print('CNN Running....')
# multi_conv_model = model_multi_conv(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt", continue_train=continue_train)


GRU Running....


## core / window.py / 

In [697]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return float(nse3), float(np.abs(pbias3)), pred_day, labels
    

In [680]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
47359,-0.712965,-0.415350,0.396062,1.516008,-0.127066,-0.343998,-0.282215,-0.227501,1.366020e+00,-0.366024,0.767534,-1.135437,0.882598,1.560221,0.567061,0.405963,1.009744,1.455259,-0.108759,0.189712,1.366020e+00,-0.366024,0.767534,-1.135437
47360,-0.679316,-0.415350,0.453062,1.516008,-0.003053,-0.359798,-0.290144,-0.222099,1.224740e+00,-0.707104,0.766696,-1.136009,0.883685,1.560221,0.586061,0.467632,1.071751,1.449019,-0.158318,0.182288,1.224740e+00,-0.707104,0.766696,-1.136009
47361,-0.671718,-0.360468,0.494227,1.516008,-0.044374,-0.335661,-0.290144,-0.010419,9.999963e-01,-0.999996,0.765858,-1.136581,0.916247,1.571192,0.567061,0.467632,1.051065,2.472402,-0.075059,0.120031,9.999963e-01,-0.999996,0.765858,-1.136581
47362,-0.670633,-0.283645,0.519561,1.516008,-0.044374,-0.208362,-0.290144,-0.221246,7.071041e-01,-1.224740,0.765019,-1.137153,0.934698,1.560221,0.595561,0.467632,1.051065,2.243588,0.056767,0.085137,7.071041e-01,-1.224740,0.765019,-1.137153
47363,-0.694512,-0.283645,0.548061,1.516008,0.017633,-0.146810,-0.381332,-0.035343,3.660240e-01,-1.366020,0.764180,-1.137723,0.990054,1.648029,0.589226,0.565275,1.113072,2.346750,-0.322853,-0.042347,3.660240e-01,-1.366020,0.764180,-1.137723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54119,-1.114569,-0.415350,0.665860,-0.272398,0.178851,-0.083560,-1.182202,-0.157422,-3.660240e-01,1.366020,1.230111,0.666578,-0.855154,-0.272674,1.296340,-0.511366,1.392103,-0.274932,-0.191027,-0.129422,-3.660240e-01,1.366020,1.230111,0.666578
54120,-1.114569,-0.415350,0.668076,-0.272398,0.133363,-0.142442,-1.182202,-0.161559,-9.065288e-12,1.414208,1.230570,0.665675,-0.867094,-0.283645,1.292221,-0.518046,1.427249,-0.150129,-0.421971,-0.127619,-9.065288e-12,1.414208,1.230570,0.665675
54121,-1.114569,-0.415350,0.663960,-0.272398,0.154048,-0.026375,-1.182202,-0.161559,3.660240e-01,1.366020,1.231027,0.664771,-0.875777,-0.283645,1.302356,-0.519074,1.379702,-0.170722,-0.438821,-0.118817,3.660240e-01,1.366020,1.231027,0.664771
54122,-1.114569,-0.415350,0.660795,-0.272398,0.168508,0.151694,-1.182202,-0.161559,7.071041e-01,1.224740,1.231485,0.663867,-0.892059,-0.283645,1.299821,-0.519074,1.406588,-0.000990,-0.183097,-0.093468,7.071041e-01,1.224740,1.231485,0.663867


In [681]:
(0.5545867681503296 + 0.497572660446167 +0.43856728076934814)/3

0.49690890312194824

In [698]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(model=gru_model,
                                                      df=test_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['GRU'])
print(val_pbias['GRU'])

(13532, 24)
5
(552, 168, 24)
(552, 120, 1)
(552, 120, 1)
predictions.shape = (552, 120, 1)
pred_day.shape = (552, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 1.3656895
nse3 = [0.50746226]
label_day[predict_day].shape = (5, 1)
mean = 1.3621424
0.5074622631072998
0.8647304773330688


In [243]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(model=gru_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 3)

print(val_nse['GRU'])
print(val_pbias['GRU'])

(13532, 24)
7
(552, 168, 24)
(552, 120, 1)
(552, 120, 1)
predictions.shape = (552, 120, 1)
pred_day.shape = (552, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 3.0423536
nse3 = [0.07262248]
label_day[predict_day].shape = (5, 1)
mean = 3.0509417
0.07262247800827026
25.678630828857422


In [310]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(47359, 24)
5
(1962, 168, 24)
(1962, 120, 1)
(1962, 120, 1)
predictions.shape = (1962, 120, 1)
pred_day.shape = (1962, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 1.3951054
nse3 = [0.6798434]
label_day[predict_day].shape = (5, 1)
mean = 1.3957


0.6798434257507324
0.9608235955238342


In [311]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(67656, 24)
5
(2807, 168, 24)
(2807, 120, 1)
(2807, 120, 1)
predictions.shape = (2807, 120, 1)
pred_day.shape = (2807, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 1.397043
nse3 = [0.6495915]
label_day[predict_day].shape = (5, 1)
mean = 1.3966885


0.6495915055274963
0.6119622588157654


In [ ]:
val_nse['LSTM'], val_pbias['LSTM'], pred, label = compa(model=elman_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['LSTM'])
print(val_pbias['LSTM'])

In [ ]:
val_nse['CNN'], val_pbias['CNN'], pred, label = compa(model=multi_conv_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['CNN'])
print(val_pbias['CNN'])

In [ ]:
x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '['+start_year+','+end_year+']' + rnn_target_column)
# plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(val_nse.keys(), val_nse.values(), 0.3, label='NSE')
# plt.xticks(x,val_nse.keys(), rotation=45)
_ = plt.legend()
plt.show()